In [19]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
# Load the training data
train_data = pd.read_csv('train.csv')
X_train = train_data.drop('label', axis=1).values
Y_train = train_data['label'].values

In [21]:
# Determine the number of classes dynamically
NumClasses = train_data['label'].nunique()

In [22]:
# Ensure the number of features matches the input shape expected by the model
input_shape = X_train.shape[1]

In [23]:
# Load the testing data
test_data = pd.read_csv('test.csv')
X_test = test_data.values  # Use all columns in the testing data

In [24]:
# Normalize pixel values to be between 0 and 1
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [25]:
# Ensure the number of features matches the input shape expected by the model
if X_test.shape[1] != input_shape:
    raise ValueError(f"Number of features in testing data ({X_test.shape[1]}) does not match the expected input shape ({input_shape}).")

In [26]:
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)

In [27]:
# Reshape the data for CNN input
X_train = X_train.reshape(-1, 32, 32, 3)
X_test = X_test.reshape(-1, 32, 32, 3)

In [28]:
# Split the training data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [32]:
# Model definition
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(NumClasses, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1024)             

In [33]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
# Training the model
epochs = 10
model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val))

Epoch 1/10


1250/1250 [==============================] - 40s 31ms/step - loss: 1.6900 - accuracy: 0.3861 - val_loss: 1.4839 - val_accuracy: 0.4718
Epoch 2/10
1250/1250 [==============================] - 36s 29ms/step - loss: 1.3900 - accuracy: 0.5046 - val_loss: 1.3325 - val_accuracy: 0.5261
Epoch 3/10
1250/1250 [==============================] - 39s 31ms/step - loss: 1.2635 - accuracy: 0.5515 - val_loss: 1.2174 - val_accuracy: 0.5731
Epoch 4/10
1250/1250 [==============================] - 40s 32ms/step - loss: 1.1725 - accuracy: 0.5865 - val_loss: 1.1631 - val_accuracy: 0.5963
Epoch 5/10
1250/1250 [==============================] - 39s 31ms/step - loss: 1.1033 - accuracy: 0.6111 - val_loss: 1.1534 - val_accuracy: 0.5976
Epoch 6/10
1250/1250 [==============================] - 39s 32ms/step - loss: 1.0406 - accuracy: 0.6310 - val_loss: 1.1247 - val_accuracy: 0.6131
Epoch 7/10
1250/1250 [==============================] - 38s 31ms/step - loss: 0.9875 - accuracy: 0.6536 - val_loss: 1.1497

In [37]:
test_loss, test_acc = model.evaluate(X_val, Y_val, verbose=2)
print('\nTest accuracy: ', test_acc)
print('\nTest Loss: ',test_loss)

313/313 - 2s - loss: 1.1413 - accuracy: 0.6135 - 2s/epoch - 8ms/step

Test accuracy:  0.6134999990463257

Test Loss:  1.1413075923919678
